## Database Builder and Extractor for DuckDB


In [2]:
import os

import duckdb
import pandas as pd


## directory structure

In [3]:
# the local data you have cloned the workshop repo to
data_directory = "/Users/d3y010/repos/github/workshop/data"

# path to where you want to create or the current database
db_file = os.path.join(data_directory, "data.duckdb")

# path to files used to build the input data
tbl_reviewer_file = os.path.join(data_directory, "tbl_reviewers.csv")
tbl_source_file = os.path.join(data_directory, "tbl_source.csv")


## define table schema

In [4]:
tbl_schema = """
CREATE TABLE tbl_log
(
reviewer_id INTEGER, 
document_id INTEGER, 
conflict VARCHAR
);

CREATE TABLE tbl_response
(
reviewer_id INTEGER, 
reviewer_name VARCHAR,
document_id INTEGER,
alignment INTEGER,
science INTEGER,
benefits INTEGER,
comments VARCHAR,
screening_order INTEGER
);

CREATE TABLE tbl_reviewer
(
reviewer_id INTEGER, 
reviewer_name VARCHAR,
);

CREATE TABLE tbl_source
(
document_id INTEGER,
submission_timestamp TIMESTAMP,
email VARCHAR,
first_name VARCHAR,
last_name VARCHAR,
phone VARCHAR,
institution VARCHAR,
authors VARCHAR,
title VARCHAR,
abstract VARCHAR,
biosketch VARCHAR,
leverage_plan VARCHAR,
student VARCHAR,
early_career VARCHAR,
registration_waiver VARCHAR,
travel_award VARCHAR,
poster_competition VARCHAR,
breakout_sessions VARCHAR
);

"""


## build database

In [5]:
with duckdb.connect(db_file) as con:
    cursor = con.cursor()
    con.sql(tbl_schema)


## check build

In [6]:
# create a connection to a file
with duckdb.connect(db_file) as con:

    cursor = con.cursor()

    cursor.execute("SHOW TABLES")

    tables = cursor.fetchall()
    for table in tables:
        print(table[0])


tbl_log
tbl_response
tbl_reviewer
tbl_source


In [7]:
with duckdb.connect(db_file) as con:
    cursor = con.cursor()

    sql = "DESCRIBE tbl_response;"
    cursor.execute(sql)
    x = cursor.fetchall()
    
x


[('reviewer_id', 'INTEGER', 'YES', None, None, None),
 ('reviewer_name', 'VARCHAR', 'YES', None, None, None),
 ('document_id', 'INTEGER', 'YES', None, None, None),
 ('alignment', 'INTEGER', 'YES', None, None, None),
 ('science', 'INTEGER', 'YES', None, None, None),
 ('benefits', 'INTEGER', 'YES', None, None, None),
 ('comments', 'VARCHAR', 'YES', None, None, None),
 ('screening_order', 'INTEGER', 'YES', None, None, None)]

## add starter data from CSV files

In [8]:
df_source = pd.read_csv(tbl_source_file)
df_reviewer = pd.read_csv(tbl_reviewer_file)

with duckdb.connect(db_file) as con:
    
    cursor = con.cursor()

    con.execute("INSERT INTO tbl_source SELECT * FROM df_source")
    con.execute("INSERT INTO tbl_reviewer SELECT * FROM df_reviewer")



## view data

In [ ]:
with duckdb.connect(db_file) as con:
    
    cursor = con.cursor()

    x = con.execute("SELECT * FROM tbl_source").df()

x


## dump database contents to file

In [11]:
with duckdb.connect(db_file) as con:
    
    cursor = con.cursor()

    cursor.execute("SHOW TABLES")

    tables = cursor.fetchall()
    for table in tables:
        
        # dumped db to parquet file
        dumped_db_file = os.path.join(data_directory, f"data_{table[0]}.parquet")
        
        con.execute(f"""
            COPY (SELECT * FROM {table[0]}) TO '{dumped_db_file}' (FORMAT 'parquet')
        """)
